# BERT

## Dataset creation

In [49]:
import pandas as pd
import re, random
import kagglehub
from datasets import load_dataset
import torch

In [50]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [51]:
# path = kagglehub.dataset_download("antonoof/avito-data")

# print("Path to dataset files:", path)

In [52]:
# avito_df = pd.read_parquet("/root/.cache/kagglehub/datasets/antonoof/avito-data/versions/1/data/train_part_0001.snappy.parquet")

In [53]:
# avito_df = avito_df.drop_duplicates(subset=['base_item_id'])

In [54]:
news_ds = load_dataset("zloelias/lenta-ru")

In [55]:
# let's create 30k sentences out of news dataset (and 5k sentences for validation)
sentences = []
for news in news_ds["train"]["text"]:
    cur_sentences = news.split(".")
    sentences.extend(cur_sentences)
    if len(sentences) > 35000:
        break

In [56]:
def create_target(text):
    letters = list(text)
    spaces_pos = [i for i,letter in enumerate(letters) if letter==" "]
    if len(spaces_pos) == 0:
        return text
    k = random.randint(min(4,len(spaces_pos)), min(15,len(spaces_pos)))
    drop_idx = random.sample(spaces_pos, k)
    return "".join(letter for i,letter in enumerate(letters) if i not in drop_idx)

In [57]:
modified_texts = list(map(create_target, sentences))

In [58]:
data_dict = {
    "input_text": modified_texts,
    "target_text": sentences,
}

dataset = Dataset.from_dict(data_dict)

In [59]:
dataset = dataset.train_test_split(test_size=0.1)

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-small").to(device)

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [62]:
dataset['train'][0]

{'input_text': 'АктрисаХалле Беррисыграетвфильме«ЛюдиИкс:Дниминувшегобудущего»',
 'target_text': 'Актриса Халле Берри сыграет в фильме «Люди Икс: Дни минувшего будущего»'}

In [63]:
def preprocess_function(examples):
    model_inputs = tokenizer(["fix spaces: " + text for text in examples["input_text"]], max_length=256, truncation=True)
    labels = tokenizer(examples["target_text"], max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [64]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/31501 [00:00<?, ? examples/s]

Map:   0%|          | 0/3501 [00:00<?, ? examples/s]

In [65]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [66]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-spaces",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    eval_steps=500,
    eval_strategy="steps",
    # per_device_eval_batch_size=16,
    warmup_steps=500,
    logging_steps=250,
   # predict_with_generate=True,
    logging_dir="./logs",
)

In [67]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-2429944011.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [68]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.862800,0.440156
1000,0.611500,0.323320
1500,0.544900,0.281183
2000,0.487600,0.252818
2500,0.442700,0.231865
3000,0.421400,0.220018
3500,0.406200,0.209949
4000,0.387000,0.202567
4500,0.371900,0.195191
5000,0.370800,0.191709


TrainOutput(global_step=7876, training_loss=0.4574977037808567, metrics={'train_runtime': 1826.847, 'train_samples_per_second': 68.973, 'train_steps_per_second': 4.311, 'total_flos': 3103341102950400.0, 'train_loss': 0.4574977037808567, 'epoch': 4.0})

In [84]:
text = "fix spaces: Тынеможешьздесьспать,"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=256, do_sample=False, num_beams=4)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Ты не можешь здесь спать,


In [86]:
for t in texts["text_no_spaces"]:
    text = "fix spaces: " + t
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=256, do_sample=False, num_beams=4)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

куплю йфон 14 про
ищу дом в Подмосковье
Сдаю квартиру м и техникой
новый диван доставка не дорого
отдамдаром кошку
Какая работа в Москве удаленно
куплю телевизор Pilips
ищу грузчиков для переезда
ремонт квартир
куплю ноутбук HP
ищу квартиру у уетро
новая микроволновка Amsung
срочно продам велосипед
куплю гитару Fender
ищу преподавателя биологии
сда югаражна длительный срок
Куплю ванбу
ищу мастера по ремонту холодильников
новый шкаф доставка сегодня
куплю Xbox One
Я ищу по выходу вчерам
Сдам комнатудентке
куплюс куплюстару юлангинигу
ищу сначала брадор
новыйтелефон Xiaomi 13
Куплю Playstation 5диск
ищу место в центр его рода
срочный подход на няня ребёнку
Куплю стиральную машину Indesit
ищу детскую кроваткубу
новая куртка доставка
Куплю велосипед Merida
ищу вратофтальмолога
с даюквартиру в центре Москвы
куплю холодильник Samsung
ищу кошку британскую
новый ноутбук доставка завтра
Куплю шкафбу
ищу репетитора по профессии
сдаю комнату без посредников
куплю аудитор LGOLED
ищу квартиру недор

KeyboardInterrupt: 

In [ ]:
#некст идея - обучить на выходи биграмм - должно начать рабоать лучше по идее